In [ ]:
import os
import numpy as np
import tensorflow as tf
from google.colab import drive
from tqdm import tqdm

print(f"TensorFlow Version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

TensorFlow Version: 2.19.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
print("\nMounting Google Drive...")
drive.mount('/content/drive')


Mounting Google Drive...
Mounted at /content/drive


In [ ]:
DRIVE_PATH = "/content/drive/My Drive/"
ZIP_FILE_PATH = os.path.join(DRIVE_PATH, "final_data.zip")
MODEL_1_PATH = os.path.join(DRIVE_PATH, "keras_cnn_model_1.keras")
MODEL_2_PATH = os.path.join(DRIVE_PATH, "keras_cnn_model_2.keras")
MODEL_3_PATH = os.path.join(DRIVE_PATH, "keras_cnn_model_3.keras")

In [ ]:
print(f"\nCopying dataset from {ZIP_FILE_PATH}...")
!cp "{ZIP_FILE_PATH}" /content/
print("Unzipping local dataset...")
!unzip -o /content/final_data.zip -d /content/ > /dev/null
print("Dataset ready on local disk.")


Copying dataset from /content/drive/My Drive/final_data.zip...
Unzipping local dataset...
Dataset ready on local disk.


In [ ]:
print("\n--- Building Test Data Pipeline ---")

IMG_SIZE = 224
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE
TEST_DIR = 'final_data/test/'

def parse_image(filename, label):
    image = tf.io.read_file(filename)
    image = tf.io.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = image / 255.0
    return image, label

test_files = []
test_labels = []
for age_folder in sorted(os.listdir(TEST_DIR)):
    folder_path = os.path.join(TEST_DIR, age_folder)
    if os.path.isdir(folder_path):
        for img_name in os.listdir(folder_path):
            test_files.append(os.path.join(folder_path, img_name))
            test_labels.append(int(age_folder))

test_ds = tf.data.Dataset.from_tensor_slices((test_files, test_labels))
test_ds = test_ds.map(parse_image, num_parallel_calls=AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)
print(f"Test dataset created with {len(test_files)} images.")


--- Building Test Data Pipeline ---
Test dataset created with 47567 images.


In [ ]:
model_1 = tf.keras.models.load_model(MODEL_1_PATH)
print("Model 1 loaded successfully.")

model_2 = tf.keras.models.load_model(MODEL_2_PATH)
print("Model 2 loaded successfully.")

model_3 = tf.keras.models.load_model(MODEL_3_PATH)
print("Model 3 loaded successfully.")

models = [model_1, model_2, model_3]

Model 1 loaded successfully.
Model 2 loaded successfully.
Model 3 loaded successfully.


In [ ]:
all_predictions = []
all_true_labels = []

print("\nRunning evaluation on the unseen test set...")
for images, labels in tqdm(test_ds, desc="Evaluating Ensemble"):
    preds1 = model_1.predict(images, verbose=0)
    preds2 = model_2.predict(images, verbose=0)
    preds3 = model_3.predict(images, verbose=0)

    avg_preds = (preds1 + preds2 + preds3) / 3.0

    all_predictions.extend(avg_preds.squeeze())
    all_true_labels.extend(labels.numpy())


Running evaluation on the unseen test set...


Evaluating Ensemble: 100%|██████████| 1487/1487 [07:23<00:00,  3.36it/s]


In [ ]:
final_mae = tf.keras.losses.MAE(all_true_labels, all_predictions).numpy()

print("\n" + "="*50)
print(f"        FINAL ENSEMBLE PROJECT RESULT")
print("="*50)
print(f"\n  Mean Absolute Error (MAE) on the unseen test set:")
print(f"  --> {final_mae:.2f} years <--")
print("\n"+"="*50)


        FINAL ENSEMBLE PROJECT RESULT

  Mean Absolute Error (MAE) on the unseen test set:
  --> 10.23 years <--



In [ ]:
import google.colab.files as files

def predict_age_for_image(image_path, models):
    try:
        image = tf.io.read_file(image_path)
        image = tf.io.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
        image = image / 255.0
        image = tf.expand_dims(image, axis=0)

        individual_preds = []
        for model in models:
            pred = model.predict(image, verbose=0).squeeze()
            individual_preds.append(pred)

        ensemble_pred = np.mean(individual_preds)

        return individual_preds, ensemble_pred

    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None, None

print("Please upload the image you want to predict the age for.")
uploaded = files.upload()

if uploaded:
    image_filename = list(uploaded.keys())[0]
    image_path_input = os.path.join("/content/", image_filename)
    if os.path.exists(image_path_input):
        individual_predictions, ensemble_prediction = predict_age_for_image(image_path_input, models)

        if individual_predictions is not None:
            print("\n--- Prediction Results ---")
            for i, pred in enumerate(individual_predictions):
                print(f"Model {i+1} Predicted Age: {pred:.2f} years")

            print(f"\nEnsemble Predicted Age: {ensemble_prediction:.2f} years")
            print("------------------------")

        os.remove(image_path_input)
        print(f"Removed uploaded file: {image_path_input}")
else:
    print("No file was uploaded.")

Please upload the image you want to predict the age for.
